DB連線-資料上傳

In [ ]:
## file
import pandas as pd


df_file = pd.read_excel('./file/20250224-C.xlsx',header = None)
# 確認日期
date = df_file.iat[1,1]
date = date.replace('日期:','').replace('年','/').replace('月','/').replace('日','')
date_list = date.split('/')
year=str(int(date_list[0])+1911)
date = year+'/'+date_list[1]+'/'+date_list[2]

df_file = df_file.iloc[4:,1:]




In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)

connectionString

In [ ]:

SQL_STATEMENT = """
INSERT INTO ConvertibleBondDaily 
(dataDate, id, name, trade_type, closing_price, change_price, open_price, high_price, low_price,
    trade_count, unit_count, total_amount, avg_price, next_ref_price, next_limit_up, next_limit_down) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""




for index, row in df_file.iterrows():
    print(index,": ",sep="")
    if not (pd.isnull(row[1])):
        if (row[1] == "合計"):
            break
        try:
            cursor = conn.cursor()
            cursor.execute(SQL_STATEMENT,
            date,
            row[1],  # id
            row[2] if pd.notnull(row[2]) else None,  # name
            row[3] if pd.notnull(row[3]) else None,  # trade_type
            float(row[4]) if pd.notnull(row[4]) else None,  # closing_price
            float(row[5]) if pd.notnull(row[5]) else None,  # change_price
            float(row[6]) if pd.notnull(row[6]) else None,  # open_price
            float(row[7]) if pd.notnull(row[7]) else None,  # high_price
            float(row[8]) if pd.notnull(row[8]) else None,  # low_price
            int(row[9]) if pd.notnull(row[9]) else None,  # trade_count
            int(row[10]) if pd.notnull(row[10]) else None,  # unit_count
            int(row[11]) if pd.notnull(row[11]) else None,  # total_amount
            float(row[12]) if pd.notnull(row[12]) else None,  # avg_price
            float(row[13]) if pd.notnull(row[13]) else None,  # next_ref_price
            float(row[14]) if pd.notnull(row[14]) else None,  # next_limit_up
            float(row[15]) if pd.notnull(row[15]) else None  # next_limit_down
            )
            print("INSERT SUCCEED")
            conn.commit()
        except Exception as e:
            print(f"Error encountered while processing row {index}")
            print(f"Row data: {row}")
            print(f"Error: {str(e)}")
            break
        

conn.commit()
conn.close()

工作日

In [ ]:
# python 3.x
import datetime
import pandas as pd

start = datetime.datetime.strptime("2021/01/01", "%Y/%m/%d")
end = datetime.datetime.strptime("2030/12/31", "%Y/%m/%d")
# date = pd.date_range(start, end)
# day = pd.date_range(start, end)
# date=date.strftime("%Y/%m/%d")
# day=day.strftime("%w")

# df_date = pd.DataFrame(date, columns=['Date'])
# df_day = pd.DataFrame(day, columns=['DayOfWeek'])

# # 使用 pd.concat() 橫向合併
# df = pd.concat([df_date, df_day], axis=1)


In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)

SQL_STATEMENT = """
INSERT INTO Date 
(Date,Day) 
VALUES (?,?)
"""




for index, row in df.iterrows():
    print(index,": ",sep="")
    try:
        cursor = conn.cursor()
        cursor.execute(SQL_STATEMENT,
        row[0],
        row[1])
        print("INSERT SUCCEED")
        conn.commit()
    except Exception as e:
        print(f"Error encountered while processing row {index}")
        print(f"Row data: {row}")
        print(f"Error: {str(e)}")
        break
    

conn.commit()

In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)

sql_query = """
                    SELECT 
            dataDate, 
            id, 
            name, 
            trade_type, 
            closing_price, 
            change_price, 
            open_price, 
            high_price, 
            low_price,
            trade_count, 
            unit_count, 
            total_amount, 
            avg_price, 
            next_ref_price, 
            next_limit_up, 
            next_limit_down
        FROM ConvertibleBondDaily
        """
        

df = pd.read_sql(sql_query,conn)


    
result = """<div class="row">
                <div class="col-md-12 mb-lg-0 mb-4">
                    <div class="card">
                        <div class="card-header p-0 position-relative mt-n4 mx-3 z-index-2">
                            <div class="row">
                                    <div class="bg-gradient-dark shadow-dark border-radius-lg pt-4 pb-3">
                                        <h6 class="text-white text-capitalize ps-3">Search Results</h6>
                                    </div>
                            </div>
                        </div>
                        
                        <div class="card-body px-0 pb-2">
                            <div class="table-responsive">
                                <table class="table align-items-center mb-0 table-hover border border-secondary border-start-0 border-end-0">
                                    <thead>
                                        <tr>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                日期</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                代號</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                名稱</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                交易</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                收市</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                漲跌</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                開市</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                最高</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                最低</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                筆數</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                單位</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                金額</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                均價</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                明日參價</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                明日漲停</th>
                                            <th
                                                class="text-center text-uppercase text-secondary text-s opacity-7">
                                                明日跌停</th>
                                        </tr>
                                    </thead>
                                    <tbody class="table-light border border-secondary border-start-0 border-end-0">
                                        """
for i in range(0,len(df)):
  #日期
  result = result+"""
                  <tr>
                                          <td>
                                              <div class="align-middle text-center text-s">
                                                  <span class="text-xs font-weight-bold">
                      """+ str(df.iloc[i]["dataDate"]) + """</span>
                                            </div>
                                        </td>"""
  
  #代號
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["id"]) + """</span>
                                                </div>
                                            </td>"""
  
  #名稱
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["name"]) + """</span>
                                                </div>
                                            </td>"""
  #收市
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["trade_type"]) + """</span>
                                                </div>
                                            </td>"""

  #收市
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["closing_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #漲跌
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["change_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #開市
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["open_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #最高
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["high_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #最低
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["low_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #筆數
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                                                </div>
                                            </td>"""
  
  #單位
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["unit_count"]) + """</span>
                                                </div>
                                            </td>"""
  
  #金額
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["total_amount"]) + """</span>
                                                </div>
                                            </td>"""
  
  #均價
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["avg_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #明日參價
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["next_ref_price"]) + """</span>
                                                </div>
                                            </td>"""
  
  #明日漲停
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["next_limit_up"]) + """</span>
                                                </div>
                                            </td>"""
  
  #明日跌停
  result = result+"""
                  <td>
                                                <div class="align-middle text-center text-s">
                                                    <span class="text-xs font-weight-bold">
                  """+ str(df.iloc[i]["next_limit_down"]) + """</span>
                                                </div>
                                            </td>"""
  
result = result+ """</tr>
                                    </tbody>
                                </table>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
"""

print(result)

In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)
try:
    sql_query = """                                                
                        SELECT Top(1) [Date]
                        FROM [ReportDb].[dbo].[Date]
                        Where [Day] <>  '0' and [Day] <> '6' and [Date] <= GETDATE()
                        ORDER BY [Date] DESC
                        """

    df = pd.read_sql(sql_query, conn)
    result = df["Date"][0]
    print(str(result))

        
except Exception as e:
    print(f"Database query error: {e}")

finally:
    conn.close()